Data preparing

In [1]:
import pandas as pd
import pickle

In [69]:
import twitter

CONSUMER_KEY = "eTG832aIi0pyU6EjfoTGge6gx"
CONSUMER_SECRET = "mjvQFaovxMuRJ0U5XpdYfPsmmqiaZFNsL7g4qkk942OdDip6hG"

ACCESS_TOKEN_KEY = "780018504787488769-KVrRy17YTgqcUzrM32ykuYipvptNKvB"
ACCESS_TOKEN_SECRET = "qtZmbBAiOrzaBxwGNdlP0bfVA3cz2AWmsv0kq1jfdIOEb"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_submition.csv")
test.cls = None

In [47]:
def get_user_twts (u_id):
    kaggle_date = 1474243200
    timeline = []
    timeline = api.GetUserTimeline(user_id=u_id, count=200, max_id=777880466351357952 , include_rts=True, exclude_replies=False)
#    for i in xrange(20):
#        if len(timeline) == 0: return []
#        oldest_twt_id = timeline.pop().id
#        timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
#        if timeline[-1].created_at_in_seconds < kaggle_date:
#            print "timeline get iterations:", i
#            break
#    oldest_twt_id = timeline.pop().id
#    timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
#    if timeline[-1].created_at_in_seconds > kaggle_date: 
#        print "BAD_USER"
#        bad_users.add(u_id)
    return timeline


In [5]:
import unicodedata
import nltk
import re

wnl = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.RegexpTokenizer('[a-z]+')

def tokenize_twt(twt):
    text = twt.text
    #normalization
    text = ''.join(c for c in unicodedata.normalize('NFD', text.lower()) if not unicodedata.combining(c))
    #deleting URL's
    text = re.sub('https?://\S+', '', text)
    #deleting HTML symbols
    text = re.sub('&\w+;', '', text)    

    words = tokenizer.tokenize(text)
    tokens = [wnl.lemmatize(word) for word in words if word not in stopwords]
    return tokens

def get_user_vocab(twts):
    vocab = {}
    for twt in twts:
        tokens = tokenize_twt(twt)
        for token in tokens:
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 1
    return vocab


In [33]:
user_tokens = {}

In [34]:
bad_users = set()

In [8]:
downloaded_users = set()

In [48]:
new_downloaded_users = set()

In [65]:
from tqdm import tqdm

In [70]:
from twitter import TwitterError

user_ids = pd.concat([train, test])['uid']
for uid in tqdm(user_ids):
    if uid in downloaded_users: continue
    if len(user_tokens) >= 1000:
        print 'TRYING TO SAVE PICKLE'
        with open('user_tokens_'+str(len(downloaded_users))+'.pkl', 'wb') as f:
            pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
            print "pickle dumped", 'user_tokens_'+str(len(downloaded_users))+'.pkl'
            user_tokens = {}
    try:    
        twts = get_user_twts(uid)
    except TwitterError:
        twts = []
        print 'error'
        next
    vocab = get_user_vocab(twts)
    user_tokens[uid] = vocab
    downloaded_users.add(uid)
    new_downloaded_users.add(uid)
    print uid, len(twts), 'downloaded', len(vocab), 'words in vocab'
print 'TRYING TO SAVE PICKLE'
with open('user_tokens_'+str(len(downloaded_users))+'.pkl', 'wb') as f:
    pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
    print "pickle dumped", 'user_tokens_'+str(len(downloaded_users))+'.pkl'
    user_tokens = {}
    

 73%|███████▎  | 8679/11947 [00:01<00:00, 5576.67it/s]

2552839785 200 downloaded 1030 words in vocab
764302356263555073 28 downloaded 215 words in vocab
4194459603 200 downloaded 490 words in vocab
385111898 199 downloaded 954 words in vocab
146107824 197 downloaded 788 words in vocab
318104157 200 downloaded 1027 words in vocab
273462904 200 downloaded 1129 words in vocab
3024288995 199 downloaded 836 words in vocab
1912457281 200 downloaded 823 words in vocab
60445589 200 downloaded 980 words in vocab
19492516 200 downloaded 889 words in vocab
1644117440 200 downloaded 568 words in vocab
567416367 0 downloaded 0 words in vocab
40725305 200 downloaded 1081 words in vocab
error
1622370690 0 downloaded 0 words in vocab
723834361963909120 199 downloaded 1253 words in vocab
614824197 198 downloaded 1071 words in vocab
42987905 200 downloaded 980 words in vocab
1584473226 200 downloaded 713 words in vocab
2833263910 0 downloaded 0 words in vocab
16904973 197 downloaded 907 words in vocab
1100042024 200 downloaded 759 words in vocab
429606970 2

 74%|███████▍  | 8847/11947 [02:55<16:02,  3.22it/s]  

3138854200 197 downloaded 668 words in vocab
351343545 200 downloaded 623 words in vocab
40556084 200 downloaded 748 words in vocab
15817928 199 downloaded 1261 words in vocab
1215109254 101 downloaded 378 words in vocab
210515538 197 downloaded 591 words in vocab
1192285790 200 downloaded 955 words in vocab
2514460219 198 downloaded 736 words in vocab
96797550 200 downloaded 1066 words in vocab
724558779006603264 197 downloaded 687 words in vocab


 74%|███████▍  | 8857/11947 [03:05<26:39,  1.93it/s]

...

2818242018 0 downloaded 0 words in vocab


100%|█████████▉| 11946/11947 [57:38<00:01,  1.30s/it]

2263956151 200 downloaded 997 words in vocab


100%|██████████| 11947/11947 [57:39<00:00,  1.30s/it]

133581466 200 downloaded 873 words in vocab


In [60]:
reload (twitter)

<module 'twitter' from '/home/dima/anaconda2/lib/python2.7/site-packages/twitter/__init__.pyc'>

In [71]:
len(user_tokens)

809

In [73]:
len(downloaded_users)

11947

In [74]:
len(new_downloaded_users)

9666

In [38]:
len(bad_users)

0

In [77]:
import time
def save_downloaded_users_set():
    t = int(time.time())
    with open(str(t)+'_downloaded_users.pkl', 'wb') as f:
        pickle.dump(downloaded_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_downloaded_users.pkl'
    return

def save_bad_users_set():
    t = int(time.time())
    with open(str(t)+'_bad_users.pkl', 'wb') as f:
        pickle.dump(bad_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_bad_users.pkl'
    return

def save_new_downloaded_users_set():
    t = int(time.time())
    with open(str(t)+'_new_downloaded_users.pkl', 'wb') as f:
        pickle.dump(new_downloaded_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_new_downloaded_users.pkl'
    return

def save_user_tokens():
    t = int(time.time())
    with open(str(t)+'_user_tokens.pkl', 'wb') as f:
        pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_user_tokens.pkl'
    return

In [76]:
save_downloaded_users_set()
save_bad_users_set()
save_new_downloaded_users_set()
save_user_tokens()

pickle dumped 1482143177_downloaded_users.pkl
pickle dumped 1482143177_bad_users.pkl
pickle dumped 1482143177_user_tokens.pkl
pickle dumped 1482143179_new_downloaded_users.pkl


In [13]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [18]:
print 'load test'

print 'downloaded', len(load_pickle('1482080100_downloaded_users.pkl'))
print 'bad', len(load_pickle('1482080101_bad_users.pkl'))
print 'tokens', len(load_pickle('1482080101_user_tokens.pkl'))

load test
downloaded 1629
bad 338
tokens 629


In [81]:
empty_test_users = load_pickle('empty_test_users.pkl')
empty_test_uids = [uid for uid in empty_test_users]

In [119]:
timeline = api.GetUserTimeline(user_id=empty_test_uids[3], count=200 , include_rts=True, exclude_replies=False)
print len(timeline), timeline[-1].id, timeline[-1].created_at

 200 810359551413387264 Sun Dec 18 05:42:27 +0000 2016
